In [8]:
import sys
from pathlib import Path
parent_dir = Path('.').resolve().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import pandas as pd
import math
import statistics
import random
import importlib

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

import math_expressions as mexpr
importlib.reload(mexpr)

import methods
importlib.reload(methods)

<module 'methods' from '/Users/preetkaria/Desktop/RA/time_to_leave/methods.py'>

In [9]:
df = pd.read_csv('../data/gen_data1.csv')

In [10]:
df.head()

,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str
0,4,6,5,5,0.133333,25,19.214557,8.736612,5,3.972425,79.509877,66.563182,1.194502,120.636589,5.483562,7.209788,28.173785338102938_28.359584955468065_17.33539...
1,3,4,5,5,0.666667,25,12.992619,5.967654,5,2.741010,46.719461,55.889114,0.835931,86.876292,26.771221,20.658119,20.390684440092205_7.781923056848202_12.083409...
2,3,4,4,5,0.066667,20,12.609502,7.525812,3,4.491680,21.694680,24.871232,0.872280,94.679091,4.865627,4.653857,15.40608457326099_6.8971592235863355_15.448465...
3,4,6,6,5,0.066667,30,19.175787,6.062907,10,1.916941,89.829010,83.239638,1.079161,141.873474,3.469631,3.908922,21.114228846495497_21.204394544100456_22.99219...
4,3,1,6,5,0.066667,10,3.557652,2.531820,2,1.801782,9.155041,9.594768,0.954170,19.701268,0.703082,0.673767,3.2132121131017914_3.800249506909465_0.9399760...


In [11]:
X = df[['N', 'n', 'h', 'c', 'mean_n', 'std_n', 'alpha_hat', 'beta_hat', 'u_star_hat']]
y = df['u_star']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)


In [12]:
model = LinearRegression()

model.fit(X_train, y_train)
display(model.score(X_train, y_train))
display(model.coef_)

0.8483290493266398

array([ 5.34835753e+00, -1.67088565e-14,  1.73369123e+01, -2.72146491e-01,
        3.39642331e+00, -3.46396372e+00, -2.40938871e-01,  1.12762178e+00,
        3.39868654e-01])

In [13]:
test_output = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=['predicted_u_star'])
test_df = test_output.merge(df, left_index=True, right_index=True)
test_df['actual_cost'] = test_df.apply(lambda row: methods.cal_cost(row['c'], row['h'], row['u'], row['predicted_u_star']), axis=1)
test_df.head()

,predicted_u_star,alpha,beta,N,n,h,c,mean_n,std_n,alpha_hat,beta_hat,u_star,u_star_hat,z,u,optimal_cost,actual_cost,intervals_str
119,-0.528795,2,1,3,5,0.666667,25,1.952246,0.410604,23,0.086360,1.807665,4.284889,0.421870,2.725384,0.611812,2.169452,2.2736876207759416_2.2746134715193658_1.269694...
77,21.502911,4,4,5,5,0.066667,25,10.663090,7.515259,2,5.296693,45.291725,21.428830,2.113588,63.560000,1.217885,2.803806,12.738778306675409_4.161169724899452_6.1996289...
148,22.143287,4,2,6,5,0.133333,30,7.793277,5.202694,2,3.473253,28.554469,19.338515,1.476560,40.804352,1.633318,2.488142,3.8704666604235474_16.86507589013137_5.0125275...
149,17.451074,2,1,6,5,0.666667,25,1.289737,0.806272,3,0.504037,6.178391,5.066895,1.219364,14.493537,5.543430,25.000000,0.8102794931369802_1.506708391891519_1.4935825...
154,10.955451,4,2,3,5,0.133333,30,10.153349,5.268330,4,2.733611,10.117193,14.585830,0.693632,29.125320,2.534417,2.422649,11.290682772360439_12.852978278417524_16.92188...


In [14]:
methods.plot_plotly(data=test_df['actual_cost'], data_label='Actual Cost')
methods.plot_plotly(data=test_df['optimal_cost'], data_label='Optimal Cost')
methods.multi_plot_plotly(data=[test_df['optimal_cost'], test_df['actual_cost']], data_label=['Optimal Cost', 'Actual Cost'])
